In [1]:
#!pip install pymssql

In [2]:
#載入標點符號
punctuation = set()
with open ('punctuation_zh_tw.txt', 'r',encoding='utf-8') as f:
    for line in f.readlines():
        punctuation.add(line.strip())
with open ('punctuation_en.txt', 'r',encoding='utf-8') as f:
    for line in f.readlines():
        punctuation.add(line.strip())
punctuation.add('\n')
punctuation.add(' ')
print(punctuation)

{')', '—', '」', '“', ',', ':', '!', '–', '✖', '...', '\n', '_', '[', '?', '〔', '︱', '❤', '～', '（', '｜', '’', '(', '、', '-', '…', '！', '‘', '．', '/', '，', '『', '【', "'", '》', ' ', '·', '』', '〕', '。', '？', '”', '《', '.', ';', '：', '@', '{', '#', '）', '"', ']', '「', '~', '】', '│', '}', '*', '；', '──', '|'}


In [3]:
#加入停用詞表
with open('stopwords_zh_tw.txt',encoding='utf-8') as file:
    stopwords = {line.strip() for line in file}

In [4]:
import jieba
#jieba.load_userdict("dict.txt.big.tw")
jieba.add_word("CP值")
jieba.add_word("必比登")
jieba.add_word("吃到飽")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ntub_pj\AppData\Local\Temp\jieba.cache
Loading model cost 1.575 seconds.
Prefix dict has been built successfully.


In [5]:
import pymssql
import pandas as pd

#取得store_info_id
with pymssql.connect(host='20.3.171.50',user='sa',password='ntub_pj123456',database='food') as conn:
    sql = "SELECT store_info_id, store_name FROM store_info"
    store_info_id = pd.read_sql(sql,conn)

store_info_id

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,store_info_id,store_name
0,1,泗元碳烤
1,2,漢坊餅藝 微風台北車站店
2,3,TINO’S PIZZA 堤諾比薩 台北濟南店
3,4,陳記京滬酒釀餅
4,5,建中黑砂糖刨冰
...,...,...
951,952,家食堂
952,953,街角柒號
953,954,中原麵店
954,955,牯嶺油飯


In [6]:
#載入全部的特別標籤
all_special_label = pd.read_csv('all_special_label.txt', sep=",",header=None)
all_special_label = all_special_label[0].values.tolist()

In [7]:
#依照store_info_id取comment
#!pip install wordcloud
from wordcloud import WordCloud
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import base64
from io import BytesIO

#先將原本的文字雲, 特別標籤清除
with pymssql.connect(host='20.3.171.50',user='sa',password='ntub_pj123456',database='food') as conn:
    with conn.cursor(as_dict=True) as cursor:
        sql = "UPDATE store_info SET wordcloud = NULL, special_label = NULL"
        cursor.execute(sql)
        conn.commit()

for store_info_id in store_info_id["store_info_id"]:
    with pymssql.connect(host='20.3.171.50',user='sa',password='ntub_pj123456',database='food') as conn:

        #取得本地評價
        sql = "SELECT store_info_id, content FROM store_local_comment where store_info_id = {}".format(store_info_id)
        local_comment = pd.read_sql(sql,conn)

        #取得外部評價
        sql = "SELECT store_info_id, content FROM store_external_comment where store_info_id = {}".format(store_info_id)
        external_comment = pd.read_sql(sql,conn)

        #合併 本地評價 + 外部評價
        all_comment = pd.concat([local_comment, external_comment], axis=0)
        print("店家id：" + str(store_info_id) + "，共" + str(len(all_comment)) + "筆評論")

        #如果沒評價則跳過
        if len(all_comment) == 0:
            print("店家id：" + str(store_info_id) + "沒評論", end = "\n\n")
            continue

        #評價輸出為pandas.txt
        all_comment = all_comment['content']
        all_comment.to_csv('pandas.txt', header=None, index=None, sep='\t')

        #開啟pandas.txt
        data={}
        with open('pandas.txt','r',encoding='utf-8') as text_file:
            text_list = text_file.read()

        #jieba分詞
        word_list = [word for word in jieba.cut(text_list, cut_all=False) if word not in punctuation and word not in stopwords]

        #依照all_special_label篩選出符合的個別special_label
        special_label = [word for word in word_list if word in all_special_label]
        special_label = list(set(special_label))

        str_special_label = ""
        for index, i in enumerate(special_label):
            if index == len(special_label)-1:
                str_special_label += i
            else:
                str_special_label += i + ","

        print("個別的special_label：" + str_special_label)

        #將special_label存入資料庫
        with conn.cursor(as_dict=True) as cursor:
            sql = "update store_info set special_label = N'{}' where store_info_id = {}".format(str_special_label, store_info_id)
            cursor.execute(sql)
            conn.commit()

        #處理輸入文字雲的data
        for word in word_list: 
            if len(word)>=2:
                if not data.__contains__(word):
                    data[word]=0
                data[word]+=1

        #顯示詞頻(依照次數排序)
        #print("詞頻：" + str(dict(sorted(data.items(), key=lambda item: item[1], reverse=True))))

        #製作文字雲
        my_wordcloud = WordCloud(  
            background_color='white',  
            max_words=60,  
            font_path=r'MicrosoftJhengHeiRegular.ttf', #字體
            #mask=imageio.imread('橢圓形.jpg'), #依照圖片形狀製作
            width=500,
            height=500
        ).generate_from_frequencies(data)

        #wordcloud去背
        image = my_wordcloud.to_image()
        image = image.convert('RGBA')
        newImage = []
        for item in image.getdata():
            if item[:3] == (255, 255, 255):
                newImage.append((255, 255, 255, 0))
            else:
                newImage.append(item)
        image.putdata(newImage)

        #wordcloud存到本地端
        #image.save("wordcloud/" + str(store_info_id) + ".png")

        #將wordcloud轉成Base64編碼
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        wordcloud_str = base64.b64encode(buffered.getvalue())
        wordcloud_str = str(wordcloud_str, 'utf-8')

        #將wordcloud存入資料庫
        with conn.cursor(as_dict=True) as cursor:
            sql = "update store_info set wordcloud = '{}' where store_info_id = {}".format(wordcloud_str, store_info_id)
            cursor.execute(sql)
            conn.commit()

        print("店家id：" + str(store_info_id) + "，完成", end = "\n\n")

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


店家id：1，共3筆評論
個別的special_label：
店家id：1，完成

店家id：2，共1筆評論
個別的special_label：


C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


店家id：2，完成

店家id：3，共5筆評論
個別的special_label：


C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


...

店家id：48，完成

店家id：49，共26筆評論
個別的special_label：平價,美味,聚餐


C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


店家id：49，完成

店家id：50，共38筆評論
個別的special_label：美味,必比登,小吃,老店


C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


KeyboardInterrupt: 